<a href="https://colab.research.google.com/github/tluxxx/General_Test/blob/main/TBackT_Draft_Version_0_2_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Draft TobisBackTrader 0.2 b

principal functionality:
1.   Preparations
2.   definition of Class TBackT, methods for trading based on signals and for calculating basic statistics


*   Trading at the end of the day where the signal was generated, execution of any order at close!!
*   LONG only and SHORT only, one instrument only
*   3 potential trading scenarios: (1) reinvest return, (2) reinvest a defined number of assets (3) reinvest a defined ammount in assets


3.   Import of Stock Data and other preparations
2.   Signal Generation 
1.   Backtesting
2.   Output of basic results (PnL-unrealized, PnL-realized) 

# 1. Preparations

In [ ]:
# colab specific preparations
!pip install yfinance
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
from google.colab import drive
drive.mount("/content/gdrive")

In [18]:
# importing modules
import pandas as pd
import numpy as np
import yfinance as yf
import talib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt
import sys
# other preparations
plt.style.use('dark_background')

#2. Definition Class TBackT

In [69]:
class TBackT:
  def __init__(self, price, direction, size_type, size, buy_sig, sell_sig, short_sig, cover_sig, init_cash=100):
    self.price = price
    self.direction = direction
    self.size_type = size_type
    self.size = size
    self.buy_sig = buy_sig
    self.sell_sig = sell_sig
    self.short_sig = short_sig
    self.cover_sig = cover_sig
    self.init_cash = init_cash
    self.position = False
    self.pos_rets = 0
    self.unrlzd = 0
    self.rlzd = 0
    self.asset_value = 0
    self.asset_flow =0

  # Trading Engine Section
  def trd(self):
    # position from signals in LONG-only mode
    if self.direction == 'LONG':
      signal_pattern = self.buy_sig.astype(int).astype(str) + self.sell_sig.astype(int).astype(str)
      signal_action = {'00':np.nan, '10':1, '01':-1, '11':0}                    # 00 -> do nothing, 10 --> buy long, 01 --> sell (= go flat), 11 --> confusion, go flat
      self.position = ((signal_pattern.map(signal_action).ffill().fillna(0) + 1)/2).replace(0.5, 0)
    # positions from signals in SHORT-only mode  
    elif self.direction == 'SHORT':
      signal_pattern = self.short_sig.astype(int).astype(str) + self.cover_sig.astype(int).astype(str)
      signal_action = {'00':np.nan, '10':-1, '01':1, '11':0}                    # 00 -> do nothing, 10 --> sell short, 01 --> cover (= go flat), 11 --> confusion, go flat
      self.position = ((signal_pattern.map(signal_action).ffill().fillna(0)-1)/2).replace(-0.5, 0)  
    # position from signals in BOTH-mode (= LONG and SHORT)
    elif self.direction == 'BOTH':
      pass
    else:
      print('something went wrong: invalid direction') 
      sys.exit(0)

  def asset_cash_calculation(self):
    # calculation from position asset values, cash and total_equity
    pos_rets = self.position.shift(1) * self.price.pct_change() 
    # size_type == reinvest  --> each time reinvesting the current cash
    if self.size_type == 'reinvest':
      if self.direction == 'LONG':
        self.asset_value = pd.Series(self.init_cash * np.where(self.position == 1, (1 + pos_rets).cumprod().fillna(1),0), index=self.position.index)  # value of asset by end of the day 
        cf = - np.where(self.position.diff() == 1, self.asset_value.diff(), np.nan)
        cf = np.where(self.position.diff() == -1, (1 + pos_rets).cumprod().fillna(1) * self.init_cash, cf)
      elif self.direction == 'SHORT':
        self.asset_value = pd.Series(self.init_cash * (-1) * np.where(self.position == -1, (1 + pos_rets).cumprod().fillna(1), 0), index=self.position.index)  # value of asset by end of the day 
        cf = - np.where(self.position.diff() == -1, -self.asset_value.diff(), np.nan)
        cf = np.where(self.position.diff() == 1, (1 + pos_rets).cumprod().fillna(1)* self.init_cash, cf)
      elif self.direction =='BOTH':
        pass
      else:
        pass
    elif self.size_type == 'ammount':
      pass
    elif self.size_type == ' value':
      pass
    else:
      print('something went wrong: invalid size_type') 
      sys.exit(0)        

    # cash_flow calculation
    cf[0] = self.init_cash
    cash_flow = pd.Series(cf, index=position.index).fillna(0)   
    self.cash = cash_flow.cumsum()
    # total equity
    self.total_equity = self.cash - self.asset_value      

  def trading(self):
    self.trd()
    self.asset_cash_calculation()


  # Plotting Section
  def position_plot(self):
    # sepration of LONG and SHORT positions 
    position_long = np.where(self.position == 1, 1, 0)
    position_short = np.where(self.position == -1, -1, 0)
    position_flat = np.where(self.position == 0, 0, np.nan)
    # visualisation of positions
    fig = go.Figure()
    fig.update_layout(xaxis_rangeslider_visible=False, width=1500, height=300, template='plotly_dark')
    fig.add_trace(go.Scatter(x=self.position.index, y=position_long, name='Position Long', fill='tozeroy', fillcolor = 'green', line_color='green'))
    fig.add_trace(go.Scatter(x=self.position.index, y=position_short, name='Position Short', fill='tozeroy', fillcolor = 'red', line_color='red'))
    fig.add_trace(go.Scatter(x=self.position.index, y=position_flat, name='Position Flat', line_color='blue'))
    fig.update_layout(title=' Positions', xaxis_title='Date', yaxis_title='Position')
    fig.update_layout(showlegend=True)
    fig.show() 

  def asset_value_plot(self):
    asset_long = self.asset_value.clip(0, None)
    asset_short = self.asset_values.clip(None, 0)
    fig = go.Figure()
    fig.update_layout(xaxis_rangeslider_visible=False, width=1500, height=300, template='plotly_dark')
    fig.add_trace(go.Scatter(x=self.asset_value.index, y=self.asset_value, name='Asset_value', fill='tozeroy', fillcolor = 'green', line_color='green'))
    fig.update_layout(title=' Asset_value', xaxis_title='Date', yaxis_title='Asset_value')
    fig.update_layout(showlegend=True)
    fig.show()   

  def total_equity_cash_plot(self):
    # plotting cash and total equity realized and unrealized PnL 
    fig = go.Figure()
    fig.update_layout(xaxis_rangeslider_visible=False, width=1500, height=300, template='plotly_dark')
    fig.add_trace(go.Scatter(x=position.index, y=cash, name='cash', fill='tozeroy', fillcolor = 'green', line_color='green'))
    fig.add_trace(go.Scatter(x=position.index, y=total_equity, name='total_equity'))
    fig.update_layout(title=' Total Equity and Cash', xaxis_title='Date', yaxis_title='Total Equity and Cash')
    fig.update_layout(showlegend=True)
    fig.show()


#3. Import StockData and preparations

In [3]:
# direct download from yfinance
start_time = '2020-01-01'
end_time = '2023-06-05'
instrument = 'BAYN.DE'
price_data = yf.download(instrument, start=start_time, end=end_time)

[*********************100%***********************]  1 of 1 completed


#4. Signal Generations

In [4]:
upper, middle, lower = talib.BBANDS(price_data.Close, timeperiod=20, nbdevup=2.5, nbdevdn=2.5, matype=0)

In [5]:
# Plotting Candlesticks + Bollinger Bands
data = [go.Candlestick(x=price_data.index, open=price_data.Open, high=price_data.High, low=price_data.Low, close=price_data.Close)]
fig = go.Figure(data)
fig.update_layout(xaxis_rangeslider_visible=False, width=1500, height=600, template='plotly_dark')
upper_trace = go.Scatter(x=upper.index, y=upper, mode='lines', name='UpBand')
middle_trace = go.Scatter(x=middle.index, y=middle, mode='lines', name='MiBand')
lower_trace = go.Scatter(x=lower.index, y=lower, mode='lines', name='LoBand')
fig.add_trace(upper_trace)
fig.add_trace(middle_trace)
fig.add_trace(lower_trace)
fig.update_layout(title_text='BAYER STOCK')
fig.show()

In [6]:
# helping functions
def crossover(x,y):
  zz = np.where((x>y) & (x.shift(1)<y.shift(1)),1,0)
  return pd.Series(zz, index=x.index)

def crossunder(x,y):
  zz = np.where((x<y) & (x.shift(1)>y.shift(1)),1,0)
  return pd.Series(zz, index=x.index)


In [7]:
# conversion in action signals --> will pe potential starting point of an backtest
buy_sig = crossover(price_data.Close, lower).fillna(0)
sell_sig = crossover(price_data.Close, middle).fillna(0)
short_sig = crossunder(price_data.Close, upper).fillna(0)
cover_sig = crossunder(price_data.Close, middle).fillna(0)
results = pd.DataFrame(price_data.Close)

In [73]:
trade = TBackT(price=price_data.Close, direction='SHORT', size_type ='reinvest', size = 1, buy_sig=buy_sig, sell_sig=sell_sig, short_sig=short_sig, cover_sig=cover_sig, init_cash=100)
trade.trading()

In [74]:
trade.position_plot()

In [75]:
trade.asset_value_plot()

In [48]:
trade.total_equity_cash_plot()

In [17]:
data ={'price':price_data.Close, 'position':position, 'pos_rets': pos_rets, 'pos_rets_cum':(1 + pos_rets).cumprod().fillna(1), 'asset_value':asset_value,'cash_flow':cash_flow, 'cash':cash}
df = pd.DataFrame(data, index=position.index)
df.to_excel('/content/gdrive/My Drive/Colab Notebooks/Backtest Basics/TBackT/TBackT Draft 0 Basis Functions/Checking_TBackT_02b_2.xlsx')
df

,price,position,pos_rets,pos_rets_cum,asset_value,cash_flow,cash
Date,,,,,,,
2020-01-02,73.519997,0.0,NaN,1.000000,-0.0,100.0,100.000000
2020-01-03,72.580002,0.0,-0.0,1.000000,-0.0,0.0,100.000000
2020-01-06,71.739998,0.0,-0.0,1.000000,-0.0,0.0,100.000000
2020-01-07,72.129997,0.0,0.0,1.000000,-0.0,0.0,100.000000
2020-01-08,74.000000,0.0,0.0,1.000000,-0.0,0.0,100.000000
...,...,...,...,...,...,...,...
2023-05-26,54.400002,0.0,0.0,1.157179,-0.0,0.0,115.717872
2023-05-29,54.349998,0.0,-0.0,1.157179,-0.0,0.0,115.717872
2023-05-30,53.419998,0.0,-0.0,1.157179,-0.0,0.0,115.717872
